In [1]:
# notebook to calculate and save TEM diagnostics
# This assumes CESM(CAM) data have already been organized into zonal mean fluxes
# Uzm, THzm, VTHzm, Vzm, UVzm, UWzm, Wzm 
# note that here we are calculating the E-P fluxes on model interface levels, which is ok
# in the stratosphere but not in the troposphere.  If interested in tropospheric
# E-P flux diagnostics, make sure they have been interpolated to pressure already.

# initial notebook coding by Dan Marsh 16 Dec 2022

In [2]:
import xarray as xr
import numpy as np
from datetime import date
import matplotlib.pyplot as plt

In [3]:
from tem4cam import calc_tem

In [4]:
# open input file
# note: for processing multiple files, simpy use xr.open_mfdataset()
test_files = '/glade/scratch/hannay/archive/f.cam6_3_106.FLTHIST_v0a.ne30.dcs_non-ogw_ubcF.001/atm/hist/*h4.1997*.nc'
ds = xr.open_mfdataset(test_files)

In [5]:
#iterate over the times in a dataset

for count, value in enumerate(ds.time.values):
    if count == 0:
        print('first date', value)
        dstem0 = calc_tem(ds.squeeze().isel(time=count))
    else:
        print(count, value)
        dstem = calc_tem(ds.squeeze().isel(time=count))
        dstem0 = xr.concat([dstem0, dstem],'time')

dstem0.attrs = ds.attrs

dstem0.attrs['created'] = str(date.today())


first date 1997-02-01 00:00:00
1 1997-03-01 00:00:00
2 1997-04-01 00:00:00
3 1997-05-01 00:00:00
4 1997-06-01 00:00:00
5 1997-07-01 00:00:00
6 1997-08-01 00:00:00
7 1997-09-01 00:00:00
8 1997-10-01 00:00:00
9 1997-11-01 00:00:00
10 1997-12-01 00:00:00
11 1998-01-01 00:00:00


In [6]:
dstem0['lev']=ds['lev']
dstem0

<xarray.Dataset>
Dimensions:    (time: 12, nbnd: 2, zalat: 90, lev: 58)
Coordinates:
    zalon      float64 0.0
  * time       (time) object 1997-02-01 00:00:00 ... 1998-01-01 00:00:00
  * zalat      (zalat) float64 -89.0 -87.0 -85.0 -83.0 ... 83.0 85.0 87.0 89.0
  * lev        (lev) float64 3.018 5.445 9.087 14.25 ... 974.5 983.2 991.2 997.5
Dimensions without coordinates: nbnd
Data variables: (12/13)
    date       (time) int32 dask.array<chunksize=(1,), meta=np.ndarray>
    datesec    (time) int32 dask.array<chunksize=(1,), meta=np.ndarray>
    time_bnds  (time, nbnd) object dask.array<chunksize=(1, 2), meta=np.ndarray>
    uzm        (time, lev, zalat) float32 -0.6618 -1.621 ... -0.4683 -0.2174
    vzm        (time, lev, zalat) float32 -0.007444 -0.01387 ... -0.04584
    epfy       (time, lev, zalat) float32 -8.144 -118.8 ... 3.687e+05 5.588e+04
    ...         ...
    vtem       (time, lev, zalat) float32 0.01009 0.01458 ... -1.152 -0.6503
    wtem       (time, lev, zalat) float32 0.01359 0.006493 ... -0.001081
    psitem     (time, lev, zalat) float32 1.485e+06 1.058e+07 ... 4.348e+07
    utendepfd  (time, lev, zalat) float32 -2.174e-06 2.582e-06 ... 5.302e-05
    utendvtem  (time, lev, zalat) float32 -1.468e-06 -2.116e-06 ... -9.477e-05
    utendwtem  (time, lev, zalat) float32 7.58e-07 8.899e-07 ... -9.58e-07
Attributes:
    Conventions:       CF-1.0
    source:            CAM
    case:              f.cam6_3_106.FLTHIST_v0a.ne30.dcs_non-ogw_ubcF.001
    logname:           hannay
    host:              cheyenne1
    initial_file:      /glade/p/cesm/amwg_dev/juliob/FWsc_ne30pg3_58L_GRID_48...
    topography_file:   /glade/p/cesmdata/cseg/inputdata/atm/cam/topo/se/ne30p...
    model_doi_url:     not_set
    time_period_freq:  month_1
    created:           2023-04-13

In [8]:
# write output to a netcdf file
dstem0.to_netcdf('../output/test.TEMdiag.nc', 
                 unlimited_dims='time', 
                 mode = 'w' )

In [10]:
!ncdump -vdate,lev ../output/test.TEMdiag.nc

netcdf test.TEMdiag {
dimensions:
	time = UNLIMITED ; // (12 currently)
	nbnd = 2 ;
	zalat = 90 ;
	lev = 58 ;
variables:
	double zalon ;
		zalon:_FillValue = NaN ;
		zalon:long_name = "longitude" ;
		zalon:units = "degrees_east" ;
	double time(time) ;
		time:_FillValue = NaN ;
		time:long_name = "time" ;
		time:bounds = "time_bnds" ;
		time:units = "days since 1995-01-01" ;
		time:calendar = "noleap" ;
	int date(time) ;
		date:long_name = "current date (YYYYMMDD)" ;
		date:coordinates = "zalon" ;
	int datesec(time) ;
		datesec:long_name = "current seconds of current date" ;
		datesec:coordinates = "zalon" ;
	double time_bnds(time, nbnd) ;
		time_bnds:_FillValue = NaN ;
		time_bnds:long_name = "time interval endpoints" ;
		time_bnds:coordinates = "zalon" ;
	double zalat(zalat) ;
		zalat:_FillValue = NaN ;
		zalat:long_name = "latitude" ;
		zalat:units = "degrees_north" ;
	double lev(lev) ;
		lev:_FillValue = NaN ;
		lev:long_name = "hybrid level at midpoints (1000*(A+B))" ;
		lev:units 